# Отток клиентов

# Содержание

[Описание проекта](#0)

[Описание данных](#1)

[Шаг 1. Подготовка данных](#2)

[Шаг 2. Исследование задачи](#3)

[Шаг 3. Борьба с дисбалансом](#4)

[Шаг 4. Тестирование модели](#5)

[Общий вывод](#6)

[Чек-лист готовности проекта](#7)

# Описание проекта <a id='0'></a>

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# Описание данных <a id='1'></a>

Данные находятся в файле `/datasets/Churn.csv` (англ. «отток клиентов»). [Скачать датасет](https://code.s3.yandex.net/datasets/Churn.csv)

Признаки

* RowNumber — индекс строки в данных

* CustomerId — уникальный идентификатор клиента

* Surname — фамилия

* CreditScore — кредитный рейтинг

* Geography — страна проживания

* Gender — пол

* Age — возраст

* Tenure — сколько лет человек является клиентом банка

* Balance — баланс на счёте

* NumOfProducts — количество продуктов банка, используемых клиентом

* HasCrCard — наличие кредитной карты

* IsActiveMember — активность клиента

* EstimatedSalary — предполагаемая зарплата

Целевой признак

* Exited — факт ухода клиента

# Шаг 1. Подготовка данных <a id='2'></a>

**Загрузите и подготовьте данные. Поясните порядок действий.**

Импортируем необходимые библиотеки

In [1]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score 
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score 
from sklearn.utils import shuffle
import numpy as np
from sklearn.metrics import roc_auc_score 
from sklearn.preprocessing import StandardScaler

Загружаем данные и выводим пример строк датасета

In [2]:
df = pd.read_csv('/datasets/Churn.csv')
df.sample(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1436,1437,15646615,Muir,576,Germany,Male,28,1.0,119336.29,2,0,1,58976.85,0
4378,4379,15771087,Harrison,757,France,Female,71,0.0,88084.13,2,1,1,154337.47,0
5185,5186,15801169,Yegorova,764,Germany,Female,39,NaN,138341.51,1,1,0,50072.94,1
7384,7385,15589881,Rowe,634,France,Female,41,7.0,0.00,2,1,1,131284.93,0
5547,5548,15713845,Merrett,688,France,Male,38,7.0,148045.68,1,1,0,175479.92,1


Выводим информацию по датасету

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Удаляем колонки которые нам не понадобятся для дальнейшей работы

In [4]:
df.drop(['RowNumber','CustomerId','Surname'],axis='columns',inplace=True)
df.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9205,564,France,Male,31,0.0,110527.17,1,1,1,87060.77,0
3538,554,Germany,Female,43,2.0,120847.11,1,1,0,7611.61,1
7260,773,France,Female,41,7.0,190238.93,1,1,1,57549.65,0
5996,638,Germany,Male,62,4.0,108716.59,2,1,1,74241.09,0
6637,850,France,Male,36,3.0,0.00,2,1,0,195033.07,0


Кодируем данные по колонке 'Gender'

In [5]:
df['Gender'].replace({'Female':1,'Male':0},inplace=True)
df.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
614,688,Germany,0,26,8.0,146133.39,1,1,1,175296.76,0
2661,742,France,1,28,2.0,191864.51,1,1,0,108457.99,1
6623,581,France,0,28,3.0,104367.50,1,1,1,29937.75,0
5358,658,France,0,39,7.0,0.00,2,1,0,48378.40,0
7484,772,France,1,35,9.0,0.00,1,0,1,25448.31,0


Кодируем данные по колонке 'Geography'

In [6]:
df_numeric=pd.get_dummies(df,columns=['Geography'])
df_numeric.sample(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
6793,623,0,50,7.0,126608.37,1,0,1,645.61,1,0,1,0
3584,506,0,28,8.0,53053.76,1,0,1,24577.34,0,0,1,0
2236,632,0,41,3.0,126550.70,1,0,0,177644.52,1,0,1,0
8473,770,1,33,6.0,0.00,2,1,1,126131.90,0,1,0,0
6423,652,0,41,8.0,115144.68,1,1,0,188905.43,0,0,0,1


Чтобы устранить дамми ловушку проводим обработку данных

In [7]:
df_numeric_ohe = pd.get_dummies(df_numeric, drop_first=True)
df_numeric_ohe.sample(5)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
1333,539,1,38,8.0,82407.51,1,1,0,13123.41,0,0,1,0
3239,762,1,19,6.0,0.00,2,1,0,55500.17,0,0,0,1
6712,599,1,43,4.0,0.00,1,1,0,170347.10,0,1,0,0
6212,730,1,62,2.0,0.00,2,1,1,162889.10,0,0,0,1
7661,612,0,44,2.0,115163.38,1,1,1,97677.52,1,0,1,0


Проводим проверку на наличие пустых полей

In [8]:
df_numeric_ohe.isnull().sum()

CreditScore            0
Gender                 0
Age                    0
Tenure               909
Balance                0
NumOfProducts          0
HasCrCard              0
IsActiveMember         0
EstimatedSalary        0
Exited                 0
Geography_France       0
Geography_Germany      0
Geography_Spain        0
dtype: int64

Чтобы избежать искажения данных при обучении, удаляем строки с пустыми значениями

In [9]:
df_num_stat = df_numeric_ohe.dropna(subset=['Tenure'])

In [10]:
df_num_stat.isnull().sum()

CreditScore          0
Gender               0
Age                  0
Tenure               0
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
Geography_France     0
Geography_Germany    0
Geography_Spain      0
dtype: int64

Если объем удаленных данных не превышает 10% мы можем продолжить работу с полученным скорректированным датасетом

In [11]:
df_num_stat.shape[0]/df_numeric.shape[0]

0.9091

**Выводы**

На первом этапе выполнены следующие действия:
- импортированы необходимые для дальнейшей работы библиотеки
- загружены данные и выведены примеры строк
- выведена информация по датасету
- удалены данные, которые не несут в себе полезной нагрузки для дальнейшего обучения ('RowNumber','CustomerId','Surname')
- проведено кодирование полей 'Gender' и 'Geography'
- удалены строки с пустыми значениями, их объем не привысил 10%, значит можно работать дальше с полученными после обработки данными

# Шаг 2. Исследование задачи <a id='3'></a>

**Исследуйте баланс классов, обучите модель без учёта дисбаланса. Кратко опишите выводы.**

Проводим разделение датасеты на обучающую, валидационную и тестовую выборки в соотношении 3:1:1

In [12]:
train, valid = train_test_split(df_num_stat, test_size=0.20, random_state=12345)
train, test = train_test_split(train, test_size=0.25, random_state=12345)

features_train = train.drop(['Exited'], axis= 1)
target_train = train['Exited']

features_valid = valid.drop(['Exited'], axis= 1)
target_valid = valid['Exited']

features_test = test.drop(['Exited'], axis= 1)
target_test = test['Exited']

print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(5454, 12)
(1819, 12)
(1818, 12)


In [13]:
df_num_stat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        9091 non-null   int64  
 1   Gender             9091 non-null   int64  
 2   Age                9091 non-null   int64  
 3   Tenure             9091 non-null   float64
 4   Balance            9091 non-null   float64
 5   NumOfProducts      9091 non-null   int64  
 6   HasCrCard          9091 non-null   int64  
 7   IsActiveMember     9091 non-null   int64  
 8   EstimatedSalary    9091 non-null   float64
 9   Exited             9091 non-null   int64  
 10  Geography_France   9091 non-null   uint8  
 11  Geography_Germany  9091 non-null   uint8  
 12  Geography_Spain    9091 non-null   uint8  
dtypes: float64(3), int64(7), uint8(3)
memory usage: 807.9 KB


Проводим масштабирование исловых полей, что бы выровнять значения по ним

In [14]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
features_train.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
3706,-0.203819,1,0.471273,0.352316,0.786022,0.782369,0,0,-0.357205,0,0,1
6805,-0.357513,1,-0.384930,-1.373506,-1.230577,0.782369,1,1,-1.671048,1,0,0
4449,0.175290,0,-0.289797,-0.683177,-1.230577,0.782369,1,0,-1.119181,1,0,0
598,0.349476,1,1.708010,0.007151,1.379462,-0.914942,0,0,-1.569064,0,1,0
1845,0.902771,0,-0.289797,1.387809,-1.230577,-0.914942,0,1,1.543790,1,0,0


Проводим обучение модели Дерево решений без учета балланса классов

In [15]:
model = DecisionTreeClassifier(random_state=12345, max_depth=20, min_samples_split=30, min_samples_leaf=5)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

print(confusion_matrix(target_valid, predicted_valid))
print("F1:", f1_score(target_valid, predicted_valid))

[[1337  113]
 [ 189  180]]
F1: 0.5438066465256798


Проводим обучение модели Случайный лес без учета балланса классов

In [16]:
model = RandomForestClassifier(random_state=12345, n_estimators=20, max_depth=15)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5420240137221269


Проводим обучение модели Логистическая регрессия без учета балланса классов

In [17]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5420240137221269


**Выводы**

По итогу проведенных работ были получены следующие итоги
- данные были разделены на три выборки в соотношении 3:1:1 (тренировочная, валидационная и тестовая)
- было проведено масштабирование числовых параметров
- было проведено обучение на данных без выравнивания классов, наилучший итог был получен для Дерева решений (F1 0.5438)

# Шаг 3. Борьба с дисбалансом <a id='4'></a>

**Улучшите качество модели, учитывая дисбаланс классов. Обучите разные модели и найдите лучшую. Кратко опишите выводы.**

Проводим исследование балланса классов, сравниваем значения классов с "0" и "1"

In [18]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    print(features_ones.shape)
    print(features_zeros.shape)
    print(target_zeros.shape)
    print(target_ones.shape)
    print("Коэффициент масштабирования классов", round(len(target_zeros)/len(target_ones), 0))
    
    return features_upsampled, target_upsampled
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

(1119, 12)
(4335, 12)
(4335,)
(1119,)
Коэффициент масштабирования классов 4.0


Обучение Дерева решений после повышения классов

In [19]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

print("F1:", f1_score(target_valid, predicted_valid))
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_one_valid))

F1: 0.47849462365591394
AUC-ROC: 0.6732613774413606


Обучение Случайного леса после повышения классов

In [20]:
model = RandomForestClassifier(random_state=12345, max_depth=30, min_samples_split=11, min_samples_leaf=1)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

print("F1:", f1_score(target_valid, predicted_valid))
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_one_valid))

F1: 0.5977961432506886
AUC-ROC: 0.8517035791047566


Обучение Логистической регрессии после повышения классов

In [21]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_upsampled, target_upsampled)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

print("F1:", f1_score(target_valid, predicted_valid))
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_one_valid))

F1: 0.5977961432506886
AUC-ROC: 0.788083356695636


Проводим понижение классов, для сравнения результатов

In [22]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

Обучение Дерева решений после понижения классов

In [23]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

print("F1:", f1_score(target_valid, predicted_valid))
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_one_valid))

F1: 0.48250460405156537
AUC-ROC: 0.6981169984113634


Обучение Случайноголеса после понижения классов

In [24]:
model = RandomForestClassifier(random_state=12345, n_estimators=20, max_depth=15)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

print("F1:", f1_score(target_valid, predicted_valid))
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_one_valid))

F1: 0.5726495726495726
AUC-ROC: 0.8325642463321186


Обучение Логистической регрессии после понижения классов

In [25]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_downsampled, target_downsampled)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

print("F1:", f1_score(target_valid, predicted_valid))
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_one_valid))

F1: 0.5726495726495726
AUC-ROC: 0.7872909073918326


Обучение Дерева решений с изменением порога

In [26]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0, 0.5, 0.02):
    predicted_valid = probabilities_one_valid > threshold 
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f}".format(
        threshold, precision, recall))

print(f1_score(target_valid, predicted_valid))
print("AUC-ROC:", roc_auc_score(target_valid, probabilities_one_valid))

Порог = 0.00 | Точность = 0.470, Полнота = 0.491
Порог = 0.02 | Точность = 0.470, Полнота = 0.491
Порог = 0.04 | Точность = 0.470, Полнота = 0.491
Порог = 0.06 | Точность = 0.470, Полнота = 0.491
Порог = 0.08 | Точность = 0.470, Полнота = 0.491
Порог = 0.10 | Точность = 0.470, Полнота = 0.491
Порог = 0.12 | Точность = 0.470, Полнота = 0.491
Порог = 0.14 | Точность = 0.470, Полнота = 0.491
Порог = 0.16 | Точность = 0.470, Полнота = 0.491
Порог = 0.18 | Точность = 0.470, Полнота = 0.491
Порог = 0.20 | Точность = 0.470, Полнота = 0.491
Порог = 0.22 | Точность = 0.470, Полнота = 0.491
Порог = 0.24 | Точность = 0.470, Полнота = 0.491
Порог = 0.26 | Точность = 0.470, Полнота = 0.491
Порог = 0.28 | Точность = 0.470, Полнота = 0.491
Порог = 0.30 | Точность = 0.470, Полнота = 0.491
Порог = 0.32 | Точность = 0.470, Полнота = 0.491
Порог = 0.34 | Точность = 0.470, Полнота = 0.491
Порог = 0.36 | Точность = 0.470, Полнота = 0.491
Порог = 0.38 | Точность = 0.470, Полнота = 0.491
Порог = 0.40 | Точно

Обучение Случайного леса после изменения порога

In [27]:
model = RandomForestClassifier(random_state=12345, n_estimators=20, max_depth=16)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.36, 0.38, 0.02):
    predicted_valid = probabilities_one_valid > threshold 
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f}".format(
        threshold, precision, recall))
    print("F1:", f1_score(target_valid, predicted_valid))
    print("AUC-ROC:", roc_auc_score(target_valid, probabilities_one_valid))

Порог = 0.36 | Точность = 0.603, Полнота = 0.561
F1: 0.5814606741573034
AUC-ROC: 0.8336361087748809
Порог = 0.38 | Точность = 0.622, Полнота = 0.539
F1: 0.5776487663280117
AUC-ROC: 0.8336361087748809


Обучение логистической регрессии с изменением порога

In [28]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0.2, 0.26, 0.02):
    predicted_valid = probabilities_one_valid > threshold 
    precision = precision_score(target_valid, predicted_valid)
    recall = recall_score(target_valid, predicted_valid)

    print("Порог = {:.2f} | Точность = {:.3f}, Полнота = {:.3f}".format(
        threshold, precision, recall))
    print("F1:", f1_score(target_valid, predicted_valid))
    print("AUC-ROC:", roc_auc_score(target_valid, probabilities_one_valid))

Порог = 0.20 | Точность = 0.390, Полнота = 0.726
F1: 0.5075757575757576
AUC-ROC: 0.7881693299691618
Порог = 0.22 | Точность = 0.402, Полнота = 0.672
F1: 0.5030425963488844
AUC-ROC: 0.7881693299691618
Порог = 0.24 | Точность = 0.432, Полнота = 0.642
F1: 0.5163398692810457
AUC-ROC: 0.7881693299691618


**Выводы**

В итоге выравнивания классов были получены следующие результаты:
- коэффициент масштабирования классов 4
- при повышении классов луший рещультат был получен для случаного леса
    - F1: 0.5977961432506886
    - AUC-ROC: 0.8517035791047566
- при понижении классов лучший результат был достигнут для случайного леса
    - F1: 0.5726495726495726
    - AUC-ROC: 0.8325642463321186
- при изменении порога лучший результат опять же был получен для случайного леса
    - F1: 0.5814606741573034
    - AUC-ROC: 0.8336361087748809

# Шаг 4. Тестирование модели <a id='5'></a>

**Проведите финальное тестирование.**

In [29]:
model = RandomForestClassifier(random_state=12345, max_depth=15, min_samples_split=10, min_samples_leaf=2, n_estimators=20)
model.fit(features_upsampled, target_upsampled)
predicted_test = model.predict(features_test)
probabilities_valid = model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]

print("F1:", f1_score(target_test, predicted_test))
print("AUC-ROC:", roc_auc_score(target_test, probabilities_one_valid))

F1: 0.6035182679296347
AUC-ROC: 0.8384534615905703


**Выводы**

Для тестирования была выбрана модель случайного леса, так как при валидации на ней были получены лучшие результаты
- F1: 0.6035182679296347
- AUC-ROC: 0.8384534615905703

# Общий вывод <a id='6'></a>


В рамках проекта были получены следующие результаты
1. Были загружены и подготовлены данные датасета
2. Данные были разделены на выборки
3. Было проведено обучение без выравнивания классов
4. Был определен коэффициент для выравнивания классов выборки
5. Было проведено обучение на выборках со скорректированными классами, лучшие результаты были получены для Случайного леса
6. Было проведено тестирование на тестовой выборке и достигнуто искомое значение F1